### Import libraries

In [1]:
%load_ext autoreload
%autoreload

from os import getcwd
from os.path import join, abspath, pardir, relpath, exists

from dataclasses import dataclass, field

import pandas as pd
import numpy as np
from numpy import matrixlib as npmat
import networkx as nx
from typing import Union
import pulp as p

from IPython.display import IFrame
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Helper methods

In [2]:
# ------------------------ #
# Helper logging functions
# ------------------------ #
def print_log(text: str) -> None:
    """ Prints the log """
    print(f"[ log ]: {text}")

def print_error(text: str) -> None:
    """ Prints the error """
    print(f"[ error ]: {text}")
# -------------------------------------------------- #
# Helper functions for matrix related operations
# -------------------------------------------------- #
def graph_to_matrix(G: Union[nx.Graph, npmat.matrix]) -> npmat.matrix:
    """
    Converts a graph to a matrix
    """
    return nx.to_numpy_matrix(G) if isinstance(G, nx.Graph) else G

def matrix_to_graph(matrix: Union[nx.Graph, npmat.matrix]) -> nx.Graph:
    """
    Convert from a numpy matrix to a network graph
    """
    return nx.from_numpy_matrix(matrix) if isinstance(matrix, npmat.matrix) else matrix

def undirected_to_directed(graph: nx.Graph) -> nx.DiGraph:
    """
    Converts an undirected graph to a directed graph
    """
    di_graph = nx.DiGraph()
    di_graph.add_edges_from(graph.edges())
    return di_graph

def csv_to_matrix(csv_file: str) -> npmat.matrix:
    """
    Returns a matrix from a csv file
    """
    return npmat.asmatrix(pd.read_csv(csv_file, header=None, on_bad_lines="skip").to_numpy())

### Documentation

In [3]:
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, 'data')
data_file = join(data_dir, "data.csv")
docs_dir = join(parent_dir, 'docs')
if exists(docs_dir):
    doc_file = relpath(join(docs_dir, 'practical_works_linear_programing_v3.pdf'))
    IFrame(doc_file, width=1200, height=350)
matrix = csv_to_matrix(data_file)

#### Linear Programming via PuLP

In [4]:
# Create the variable to contain the problem data
problem = p.LpProblem(name="The Miracle Worker", sense=p.const.LpMaximize)

# Create problem variables
x = p.LpVariable(name="Medicine_1_units", lowBound=0, upBound=None, cat=p.LpInteger)
y = p.LpVariable(name="Medicine_2_units", lowBound=0, upBound=None, cat=p.LpInteger)

# The objective function is added to "problem" first
problem += 25*x + 20*y, "Health restored; to be maximized"

# The two contraints for the herbs
problem += 3*x + 4*y <= 25, "Herb A constraint"
problem += 2*x + y <= 10, "Herb B constraint"

# The problem data is written to an .lp file
problem.writeLP(filename=join(data_dir, "miracle_worker.lp"), writeSOS=1, mip=1, max_length=100)

# The problem is solved using PuLP's choice of solver
problem.solve()

/opt/homebrew/Caskroom/mambaforge/base/envs/decision_modelling/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


[Medicine_1_units, Medicine_2_units]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/mambaforge/base/envs/decision_modelling/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/671c1cc667684d3e9bb5433dc5a1749a-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/671c1cc667684d3e9bb5433dc5a1749a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 18 RHS
At line 21 BOUNDS
At line 24 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 155 - 0.00 seconds
Cgl0004I processed model has 2 rows, 2 columns (2 integer (0 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0012I Integer solution of -155 found by DiveCoefficient after 0 iterations and

1

#### Output

In [5]:
print_log(f"{p.LpStatus[problem.status] = }")

_ = [print_log(f"{v.name} = {v.varValue}") for v in problem.variables()]

print_log(f"{p.value(problem.objective) = }")

[ log ]: p.LpStatus[problem.status] = 'Optimal'
[ log ]: Medicine_1_units = 3.0
[ log ]: Medicine_2_units = 4.0
[ log ]: p.value(problem.objective) = 155.0


#### Toy example (Linear Programming via PuLP)

In [6]:
# Create the variable to contain the problem data
problem = p.LpProblem(name="Toy Manufacturing", sense=p.const.LpMaximize)

# Create problem variables
x = p.LpVariable(name="Toy_1_units", lowBound=0, upBound=None, cat=p.LpInteger)
y = p.LpVariable(name="Toy_2_units", lowBound=0, upBound=None, cat=p.LpInteger)

# The objective function is added to "problem" first
problem += 25*x + 20*y, "Profit; to be maximized"

# The two contraints for the herbs
problem += 20*x + 12*y <= 2000, "Required units - constraint"
problem += 5*x + 5*y <= 540, "Time required - constraint"

# The problem data is written to an .lp file
problem.writeLP(filename=join(data_dir, "toy_manufacturing.lp"), writeSOS=1, mip=1, max_length=100)

# The problem is solved using PuLP's choice of solver
problem.solve()

[Toy_1_units, Toy_2_units]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/mambaforge/base/envs/decision_modelling/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/658458297cea4ed3a36ffcdb30e67adf-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/658458297cea4ed3a36ffcdb30e67adf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 18 RHS
At line 21 BOUNDS
At line 24 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2600 - 0.00 seconds
Cgl0004I processed model has 2 rows, 2 columns (2 integer (0 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0012I Integer solution of -2600 found by DiveCoefficient after 0 iterations a

1

#### Output

In [7]:
print_log(f"{p.LpStatus[problem.status] = }")

_ = [print_log(f"{v.name} = {v.varValue}") for v in problem.variables()]

print_log(f"{p.value(problem.objective) = }")

[ log ]: p.LpStatus[problem.status] = 'Optimal'
[ log ]: Toy_1_units = 88.0
[ log ]: Toy_2_units = 20.0
[ log ]: p.value(problem.objective) = 2600.0


### How to visit Paris ? (efficiently with low budget)

In [8]:
@dataclass(frozen=False, order=False)
class SiteInfo:
    """
    A dataclass to hold the site information
    """
    name: str = field(default="")
    site_code: str = field(default="")
    price: float = field(default=0.0) # price in euros
    duration: float = field(default=0.0) # duration in hours
    rating: int = field(default=0) # appreciation rating

In [9]:
sites_info = [
    SiteInfo(name="La Tour Eiffel", site_code="TE", duration=4.5, rating=5, price=15.50),
    SiteInfo(name="Le Musée du louvre", site_code="ML", duration=3, rating=4, price=12),
    SiteInfo(name="l’Arc de triomphe", site_code="AT", duration=1, rating=3, price=9.50),
    SiteInfo(name="le Musée d’Orsay", site_code="MO", duration=2, rating=2, price=11),
    SiteInfo(name="le Jardin des tuileries", site_code="JT", duration=1.5, rating=3, price=0),
    SiteInfo(name="les Catacombes", site_code="CA", duration=2, rating=4, price=10),
    SiteInfo(name="le Centre Pompido", site_code="CP", duration=2.5, rating=1, price=10),
    SiteInfo(name="la Cathédrale Notre Dame de Paris", site_code="CN", duration=2, rating=5, price=5),
    SiteInfo(name="la Basilique du Sacré-Coeur", site_code="BS", duration=2, rating=4, price=8),
    SiteInfo(name="la Sainte Chapelle", site_code="SC", duration=1.5, rating=1, price=8.50),
    SiteInfo(name="La Place de la Concorde", site_code="PC", duration=0.75, rating=3, price=0),
    SiteInfo(name="la Tour Montparnasse", site_code="TM", duration=2, rating=2, price=15),
    SiteInfo(name="l’Avenue des Champs-Elysées", site_code="AC", duration=1.5, rating=5, price=0),
]

In [10]:
sites = [x.site_code for x in sites_info]

distance_in_kms = npmat.asmatrix(data=[
        [0, 3.8, 2.1, 2.4, 3.5, 4.2, 5.0,  4.4, 5.5, 4.2, 2.5, 3.1, 1.9],
        [0,   0, 3.8, 1.1, 1.3, 3.3, 1.3,  1.1, 3.4, 0.8, 1.7, 2.5, 2.8],
        [0,   0,   0, 3.1, 3.0, 5.8, 4.8,  4.9, 4.3, 4.6, 2.2, 4.4, 1.0],
        [0,   0,   0,   0, 0.9, 3.1, 2.5,  2.0, 3.9, 1.8, 1.0, 2.3, 2.1],
        [0,   0,   0,   0,   0, 4.2, 2.0,  2.4, 2.7, 2.0, 1.0, 3.4, 2.1],
        [0,   0,   0,   0,   0,   0, 3.5,  2.7, 6.5, 2.6, 3.8, 1.3, 4.9],
        [0,   0,   0,   0,   0,   0,   0, 0.85, 3.7, 0.9, 2.7, 3.4, 3.8],
        [0,   0,   0,   0,   0,   0,   0,    0, 4.5, 0.4, 2.8, 2.7, 3.9],
        [0,   0,   0,   0,   0,   0,   0,    0,   0, 4.2, 3.3, 5.7, 3.8],
        [0,   0,   0,   0,   0,   0,   0,    0,   0,   0, 2.5, 2.6, 3.6],
        [0,   0,   0,   0,   0,   0,   0,    0,   0,   0,   0, 3.0, 1.2],
        [0,   0,   0,   0,   0,   0,   0,    0,   0,   0,   0,   0, 2.1],
        [0,   0,   0,   0,   0,   0,   0,    0,   0,   0,   0,   0,   0]
    ], dtype=float)
    
distance_df = pd.DataFrame(np.matrix(distance_in_kms.T + distance_in_kms), columns=sites, index=sites)
distance_df

,TE,ML,AT,MO,JT,CA,CP,CN,BS,SC,PC,TM,AC
TE,0.0,3.8,2.1,2.4,3.5,4.2,5.00,4.40,5.5,4.2,2.5,3.1,1.9
ML,3.8,0.0,3.8,1.1,1.3,3.3,1.30,1.10,3.4,0.8,1.7,2.5,2.8
AT,2.1,3.8,0.0,3.1,3.0,5.8,4.80,4.90,4.3,4.6,2.2,4.4,1.0
MO,2.4,1.1,3.1,0.0,0.9,3.1,2.50,2.00,3.9,1.8,1.0,2.3,2.1
JT,3.5,1.3,3.0,0.9,0.0,4.2,2.00,2.40,2.7,2.0,1.0,3.4,2.1
CA,4.2,3.3,5.8,3.1,4.2,0.0,3.50,2.70,6.5,2.6,3.8,1.3,4.9
CP,5.0,1.3,4.8,2.5,2.0,3.5,0.00,0.85,3.7,0.9,2.7,3.4,3.8
CN,4.4,1.1,4.9,2.0,2.4,2.7,0.85,0.00,4.5,0.4,2.8,2.7,3.9
BS,5.5,3.4,4.3,3.9,2.7,6.5,3.70,4.50,0.0,4.2,3.3,5.7,3.8
SC,4.2,0.8,4.6,1.8,2.0,2.6,0.90,0.40,4.2,0.0,2.5,2.6,3.6


##### 1. It is assumed that Mr. Doe gives equal importance to each tourist site, and he wants to visit the maximum number of sites. Which list(s) of places could you recommend to him ? This solution will be called `ListVisit 1`.

In [16]:
# Create the variable to contain the problem data
problem = p.LpProblem(name="Paris Visit - Max. number of sites", sense=p.const.LpMaximize)

# Create the variables
for site in sites:
    site_info = next(x for x in sites_info if x.site_code == site)
    print_log(f"Creating variable for {site = }")
    globals()[f"{site}"] = p.LpVariable(name=f"{site}", lowBound=0, upBound=1, cat=p.LpInteger)

# Create the objective function
# problem += p.lpSum([globals()[f"{site}"] * sites_info[i].rating for i, site in enumerate(sites)])
problem += p.lpSum([globals()[f"{site}"] * 1 for site in sites]), "Max. number of sites"

# Create the constraints
# 1. Max. duration
problem += p.lpSum([globals()[f"{site}"] * sites_info[i].duration for i, site in enumerate(sites)]) <= 12, "Max. duration"
# 2. Max. price
problem += p.lpSum([globals()[f"{site}"] * sites_info[i].price for i, site in enumerate(sites)]) <= 65, "Max. price"

# The problem data is written to an .lp file
problem.writeLP(filename=join(data_dir, "ListVisit_1.lp"), writeSOS=1, mip=1, max_length=100)

# The problem is solved using PuLP's choice of solver
problem.solve()

[ log ]: Creating variable for site = 'TE'
[ log ]: Creating variable for site = 'ML'
[ log ]: Creating variable for site = 'AT'
[ log ]: Creating variable for site = 'MO'
[ log ]: Creating variable for site = 'JT'
[ log ]: Creating variable for site = 'CA'
[ log ]: Creating variable for site = 'CP'
[ log ]: Creating variable for site = 'CN'
[ log ]: Creating variable for site = 'BS'
[ log ]: Creating variable for site = 'SC'
[ log ]: Creating variable for site = 'PC'
[ log ]: Creating variable for site = 'TM'
[ log ]: Creating variable for site = 'AC'


[AC, AT, BS, CA, CN, CP, JT, ML, MO, PC, SC, TE, TM]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/mambaforge/base/envs/decision_modelling/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/d201e9ca95604722819bb0332c3b023b-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/d201e9ca95604722819bb0332c3b023b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 70 RHS
At line 73 BOUNDS
At line 87 ENDATA
Problem MODEL has 2 rows, 13 columns and 23 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7.875 - 0.00 seconds
Cgl0004I processed model has 2 rows, 12 columns (12 integer (11 of which binary)) and 22 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.125
Cbc0038I Pass

1

#### Output

In [25]:
print_log(f"{p.LpStatus[problem.status] = }")

# _ = [print_log(f"{v.name} = {v.varValue}") for v in problem.variables()]
to_visit = []
for v in problem.variables():
    print_log(f"{v.name} = {v.varValue}")
    if v.varValue == 1:
        site = next(x for x in sites_info if x.site_code == v.name)
        to_visit.append(site.name)

print_log(f"{p.value(problem.objective) = }")
print_log(f"You should visit total '{p.value(problem.objective)}' places. i.e:\n\n{' --- '.join(to_visit)}")

[ log ]: p.LpStatus[problem.status] = 'Optimal'
[ log ]: AC = 1.0
[ log ]: AT = 1.0
[ log ]: BS = 1.0
[ log ]: CA = 1.0
[ log ]: CN = 1.0
[ log ]: CP = 0.0
[ log ]: JT = 0.0
[ log ]: ML = 0.0
[ log ]: MO = 0.0
[ log ]: PC = 1.0
[ log ]: SC = 1.0
[ log ]: TE = 0.0
[ log ]: TM = 0.0
[ log ]: p.value(problem.objective) = 7.0
[ log ]: You should visit total '7.0' places. i.e:

l’Avenue des Champs-Elysées --- l’Arc de triomphe --- la Basilique du Sacré-Coeur --- les Catacombes --- la Cathédrale Notre Dame de Paris --- La Place de la Concorde --- la Sainte Chapelle


##### 2. Actually, Mr. Doe has some preferences among these tourist sites and he expresses them as follows:

- Preference 1 : If two sites are geographically very close (within a radius of 1 km of walking), he will prefer to visit these two sites instead of visiting only one.

In [35]:
# Create the variable to contain the problem data
problem = p.LpProblem(name="Paris Visit - Max. number of sites (within 1 km radius)", sense=p.const.LpMaximize)

# Create the variables
for site in sites:
    site_info = next(x for x in sites_info if x.site_code == site)
    print_log(f"Creating variable for {site = }")
    globals()[f"{site}"] = p.LpVariable(name=f"{site}", lowBound=0, upBound=1, cat=p.LpInteger)

# Create the objective function
problem += p.lpSum([globals()[f"{site}"] * sites_info[i].rating for i, site in enumerate(sites)]), "Max. number of sites"

# Create the constraints
# 1. Max. duration
problem += p.lpSum([globals()[f"{site}"] * sites_info[i].duration for i, site in enumerate(sites)]) <= 12, "Max. duration"
# 2. Max. price
problem += p.lpSum([globals()[f"{site}"] * sites_info[i].price for i, site in enumerate(sites)]) <= 65, "Max. price"

# 3. Distance between sites
for i, site1 in enumerate(sites):
    for j, site2 in enumerate(sites):
        if distance_df.loc[site1, site2] <= 1 and site1 != site2: # if distance is between 0 and 1 km
            print_log(f"{site1 = } {site2 = } -> {distance_df.loc[site1, site2]}")
            problem += globals()[f"{site1}"] - globals()[f"{site2}"] == 0, f"Distance between {site1} and {site2}"
            # problem += locals()[f"{site1}"] + locals()[f"{site2}"] <= 1 + distance_df.loc[site1, site2] / 10, f"Distance between {site1} and {site2}"

# The problem data is written to an .lp file
problem.writeLP(filename=join(data_dir, "ListVisit_2_a.lp"), writeSOS=1, mip=1, max_length=100)

# The problem is solved using PuLP's choice of solver
problem.solve()

[ log ]: Creating variable for site = 'TE'
[ log ]: Creating variable for site = 'ML'
[ log ]: Creating variable for site = 'AT'
[ log ]: Creating variable for site = 'MO'
[ log ]: Creating variable for site = 'JT'
[ log ]: Creating variable for site = 'CA'
[ log ]: Creating variable for site = 'CP'
[ log ]: Creating variable for site = 'CN'
[ log ]: Creating variable for site = 'BS'
[ log ]: Creating variable for site = 'SC'
[ log ]: Creating variable for site = 'PC'
[ log ]: Creating variable for site = 'TM'
[ log ]: Creating variable for site = 'AC'
[ log ]: site1 = 'ML' site2 = 'SC' -> 0.8
[ log ]: site1 = 'AT' site2 = 'AC' -> 1.0
[ log ]: site1 = 'MO' site2 = 'JT' -> 0.9
[ log ]: site1 = 'MO' site2 = 'PC' -> 1.0
[ log ]: site1 = 'JT' site2 = 'MO' -> 0.9
[ log ]: site1 = 'JT' site2 = 'PC' -> 1.0
[ log ]: site1 = 'CP' site2 = 'CN' -> 0.85
[ log ]: site1 = 'CP' site2 = 'SC' -> 0.9
[ log ]: site1 = 'CN' site2 = 'CP' -> 0.85
[ log ]: site1 = 'CN' site2 = 'SC' -> 0.4
[ log ]: site1 = 'S

/opt/homebrew/Caskroom/mambaforge/base/envs/decision_modelling/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


[AC, AT, BS, CA, CN, CP, JT, ML, MO, PC, SC, TE, TM]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/mambaforge/base/envs/decision_modelling/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/c0b7d91c613d464abc4c6f49fd90b2b0-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/c0b7d91c613d464abc4c6f49fd90b2b0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 118 RHS
At line 137 BOUNDS
At line 151 ENDATA
Problem MODEL has 18 rows, 13 columns and 55 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 25.5278 - 0.00 seconds
Cgl0004I processed model has 2 rows, 7 columns (7 integer (7 of which binary)) and 14 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.138889
Cbc003

1

#### Output

In [36]:
print_log(f"{p.LpStatus[problem.status] = }")

# _ = [print_log(f"{v.name} = {v.varValue}") for v in problem.variables()]
to_visit = []
for v in problem.variables():
    print_log(f"{v.name} = {v.varValue}")
    if v.varValue == 1:
        site = next(x for x in sites_info if x.site_code == v.name)
        to_visit.append(site.name)

print_log(f"{p.value(problem.objective) = }")
print_log(f"You should visit total '{p.value(problem.objective)}' places. i.e:\n\n{' --- '.join(to_visit)}")

[ log ]: p.LpStatus[problem.status] = 'Optimal'
[ log ]: AC = 1.0
[ log ]: AT = 1.0
[ log ]: BS = 1.0
[ log ]: CA = 1.0
[ log ]: CN = 0.0
[ log ]: CP = 0.0
[ log ]: JT = 1.0
[ log ]: ML = 0.0
[ log ]: MO = 1.0
[ log ]: PC = 1.0
[ log ]: SC = 0.0
[ log ]: TE = 0.0
[ log ]: TM = 0.0
[ log ]: p.value(problem.objective) = 24.0
[ log ]: You should visit total '24.0' places. i.e:

l’Avenue des Champs-Elysées --- l’Arc de triomphe --- la Basilique du Sacré-Coeur --- les Catacombes --- le Jardin des tuileries --- le Musée d’Orsay --- La Place de la Concorde


- Preference 2 : He absolutely wants to visit the `Eiffel Tower` (TE) and `Catacombes` (CA).

In [ ]:
# Create the variable to contain the problem data
problem = p.LpProblem(name="Paris Visit - Max. number of sites (must visit effiel tower and catacombs)", sense=p.const.LpMaximize)

# Create the variables
for site in sites:
    site_info = next(x for x in sites_info if x.site_code == site)
    print_log(f"Creating variable for {site = }")
    globals()[f"{site}"] = p.LpVariable(name=f"{site}", lowBound=0, upBound=1, cat=p.LpInteger)

# Create the objective function
problem += p.lpSum([globals()[f"{site}"] * sites_info[i].rating for i, site in enumerate(sites)]), "Max. number of sites"

# Create the constraints
# 1. Max. duration
problem += p.lpSum([globals()[f"{site}"] * sites_info[i].duration for i, site in enumerate(sites)]) <= 12, "Max. duration"
# 2. Max. price
problem += p.lpSum([globals()[f"{site}"] * sites_info[i].price for i, site in enumerate(sites)]) <= 65, "Max. price"

# 3. Must visit Effiel Tower (TE) and Catacombs (CA)
problem += globals()[f"{site1}"] - globals()[f"{site2}"] == 0, f"Distance between {site1} and {site2}"

# The problem data is written to an .lp file
problem.writeLP(filename=join(data_dir, "ListVisit_2_b.lp"), writeSOS=1, mip=1, max_length=100)

# The problem is solved using PuLP's choice of solver
problem.solve()

In [31]:
for i, site1 in enumerate(sites):
    for j, site2 in enumerate(sites):
        if distance_df.loc[site1, site2] <= 1 and site1 != site2: # if distance is between 0 and 1 km
            print_log(f"{site1 = } {site2 = } -> {distance_df.loc[site1, site2]}")

[ log ]: site1 = 'ML' site2 = 'SC' -> 0.8
[ log ]: site1 = 'AT' site2 = 'AC' -> 1.0
[ log ]: site1 = 'MO' site2 = 'JT' -> 0.9
[ log ]: site1 = 'MO' site2 = 'PC' -> 1.0
[ log ]: site1 = 'JT' site2 = 'MO' -> 0.9
[ log ]: site1 = 'JT' site2 = 'PC' -> 1.0
[ log ]: site1 = 'CP' site2 = 'CN' -> 0.85
[ log ]: site1 = 'CP' site2 = 'SC' -> 0.9
[ log ]: site1 = 'CN' site2 = 'CP' -> 0.85
[ log ]: site1 = 'CN' site2 = 'SC' -> 0.4
[ log ]: site1 = 'SC' site2 = 'ML' -> 0.8
[ log ]: site1 = 'SC' site2 = 'CP' -> 0.9
[ log ]: site1 = 'SC' site2 = 'CN' -> 0.4
[ log ]: site1 = 'PC' site2 = 'MO' -> 1.0
[ log ]: site1 = 'PC' site2 = 'JT' -> 1.0
[ log ]: site1 = 'AC' site2 = 'AT' -> 1.0


In [ ]:
# # 3. Distance between sites
# for i, site in enumerate(sites):
#     for j, site2 in enumerate(sites):
#         if i != j:
#             problem += locals()[f"{site}"] + locals()[f"{site2}"] <= 1 + distance_df.loc[site, site2] / 10, f"Distance between {site} and {site2}"

#     locals()[f"duration_{site}"] = p.LpVariable(name=f"duration_{site}", lowBound=0, upBound=site_info.duration, cat=p.const.LpContinuous)
#     locals()[f"price_{site}"] = p.LpVariable(name=f"price_{site}", lowBound=0, upBound=site_info.price, cat=p.const.LpContinuous)

#     locals()[site] = p.LpVariable(name=site, lowBound=0, upBound=1, cat=p.const.LpInteger)


# x = p.LpVariable(name="Toy_1_units", lowBound=0, upBound=None, cat=p.LpInteger)
# y = p.LpVariable(name="Toy_2_units", lowBound=0, upBound=None, cat=p.LpInteger)

# # The objective function is added to "problem" first
# problem += 25*x + 20*y, "Profit; to be maximized"

# # The two contraints for the herbs
# problem += 20*x + 12*y <= 2000, "Required units - constraint"
# problem += 5*x + 5*y <= 540, "Time required - constraint"